In [22]:
import torch
import time
n, p = 128, 4000
col = 0

In [ ]:
match_ab = torch.matmul(a, torch.t(b))
p_ab = F.softmax(match_ab, dim=1)

match_ba = torch.t(match_ab)
match_bb = torch.matmul(b, torch.t(b))
add = np.log(gamma) if gamma < 1.0 else 0.0
match_ab_bb = torch.cat([match_ba, match_bb + add], dim=1)
p_ba_bb = torch.clamp(F.softmax(match_ab_bb, dim=1), min=1e-8)
N = a.shape[0]
M = b.shape[0]
Tbar_ul, Tbar_uu = p_ba_bb[:, :N], p_ba_bb[:, N:]
I = torch.eye(M)
I = I.cuda() if Tbar_uu.is_cuda else I
middle = torch.inverse(I - Tbar_uu + 1e-8)
p_aba = torch.matmul(torch.matmul(p_ab, middle), Tbar_ul)

In [21]:
X = torch.randn([n, p]) * 0.5
# X = torch.softmax(X, 0)
B = torch.matmul(X.T, X)
B = B.cuda()
B = B.cpu()
now = time.time()
B = torch.inverse(B)
time.time() - now

1.1735286712646484

In [7]:
new_col = v = torch.randn([n, 1]) * 0.5
# new_col = v = torch.softmax(torch.randn([n, 1]) * 0.5, 0)

In [8]:
def add_new_col_to_inverse(old_inverse, new_col):
    B = old_inverse
    v = new_col
    
    u1 = torch.matmul(X.T, v)
    u2 = torch.matmul(B, u1)
    F22inv = 1. / (torch.matmul(v.T, v) - torch.matmul(u1.T, u2))
    u3 = F22inv * u2
    F11inv = B + F22inv * u2 * u2.T
    Bnew = torch.cat([
        torch.cat([F11inv, -u3], 1),
        torch.cat([-u3.T, F22inv], 1)
    ], 0)
    return Bnew

In [9]:
now = time.time()
add_new_col_to_inverse(B, v)
time.time() - now

0.338165283203125

In [29]:
Xnew = torch.cat([X, v], 1)
torch.inverse(torch.matmul(Xnew.T, Xnew))

tensor([[ 1.2506, -0.6256,  0.1035,  1.0205],
        [-0.6256,  0.8223,  0.1486, -0.5141],
        [ 0.1035,  0.1486,  0.4402,  0.3598],
        [ 1.0205, -0.5141,  0.3598,  2.0012]])